In [13]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

from lasagne.layers import InputLayer, DenseLayer, DropoutLayer
from lasagne.layers.dnn import Conv2DDNNLayer, MaxPool2DDNNLayer
from lasagne import nonlinearities
from lasagne import objectives
from lasagne import updates

from nolearn.lasagne import NeuralNet
from nolearn.lasagne.handlers import SaveWeights, PrintLayerInfo
from nolearn.lasagne.util import get_conv_infos

from nolearn_utils.iterators import (
    BufferedBatchIteratorMixin,
    ReadImageBatchIteratorMixin,
    ShuffleBatchIteratorMixin,
    LCNBatchIteratorMixin,
    RandomFlipBatchIteratorMixin,
    EqualizeAdaptHistBatchIteratorMixin,
    AffineTransformBatchIteratorMixin,
    make_iterator
)

Using gpu device 0: GeForce GTX 670


In [17]:
image_size = 28
mnist = fetch_mldata('MNIST original')
X = mnist.data.reshape(-1, image_size, image_size)
y = mnist.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
batch_size = 96
n_classes = 10

train_iterator_mixins = [
    ShuffleBatchIteratorMixin,
    RandomFlipBatchIteratorMixin,
    AffineTransformBatchIteratorMixin,
    EqualizeAdaptHistBatchIteratorMixin,
    BufferedBatchIteratorMixin,
]
TrainIterator = make_iterator('TrainIterator', train_iterator_mixins)

test_iterator_mixins = [
    EqualizeAdaptHistBatchIteratorMixin,
    BufferedBatchIteratorMixin,
]
TestIterator = make_iterator('TestIterator', test_iterator_mixins)

train_iterator_kwargs = {
    'buffer_size': 5,
    'batch_size': batch_size,
    'flip_horizontal_p': 0.5,
    'flip_vertical_p': 0.5,
    'affine_p': 0.5,
    'affine_scale_choices': np.linspace(0.75, 1.25, 5),
    'affine_translation_choices': np.arange(-24, 24, 4),
    'affine_rotation_choices': np.arange(0, 360, 36),
    'eqadapthist_ntiles_x': 16,
    'eqadapthist_ntiles_y': 16,
}
train_iterator = TrainIterator(**train_iterator_kwargs)

test_iterator_kwargs = {
    'buffer_size': 5,
    'batch_size': batch_size,
    'eqadapthist_ntiles_x': 16,
    'eqadapthist_ntiles_y': 16,
}
test_iterator = TestIterator(**test_iterator_kwargs)

net = NeuralNet(
    layers=[
        ('in', InputLayer),
        ('l1c1', Conv2DDNNLayer),
        ('l1c2', Conv2DDNNLayer),
        ('l1p', MaxPool2DDNNLayer),

        ('l7', DenseLayer),
        ('l7drop', DropoutLayer),

        ('out', DenseLayer),
    ],

    in_shape=(None, 1, image_size, image_size),

    l1c1_num_filters=32, l1c1_filter_size=(5, 5), l1c1_border_mode='same',
    l1c2_num_filters=16, l1c2_filter_size=(5, 5), l1c2_border_mode='same',
    l1p_pool_size=(3, 3),
    l1p_stride=2,

    l7_num_units=128,
    l7drop_p=0.5,

    out_num_units=n_classes,
    out_nonlinearity=nonlinearities.softmax,

    regression=False,
    objective_loss_function=objectives.categorical_crossentropy,

    update=updates.rmsprop,
    update_learning_rate=1e-3,

    eval_size=0.1,
    batch_iterator_train=train_iterator,
    batch_iterator_test=test_iterator,

    verbose=10,
    max_epochs=100
)

In [ ]:
net.fit(X_train, y_train)

/home/felixlau/devel/lasagne/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "
/home/felixlau/devel/nolearn/nolearn/lasagne/base.py:275: UserWarning: lasagne.objectives.Objective is deprecated and will be removed for the first release of Lasagne. For alternatives, please see: http://lasagne.readthedocs.org/en/latest/modules/objectives.html
  obj = objective(output_layer, **objective_params)
